In [1]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.0.1'
# spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

## import matplotlib, etc
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


Get:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:4 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:6 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:7 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:12 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  R

In [2]:
# suppress warnings, if you want:
import warnings
warnings.filterwarnings('ignore')

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.18.jar").getOrCreate()

In [4]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url="https://markdatabucket.s3.amazonaws.com/cbb.csv"

spark.sparkContext.addFile(url)
spark_data_df = spark.read.csv(SparkFiles.get("cbb.csv"), sep=",", header=True, inferSchema=True)

cbb_df = spark_data_df.toPandas()

# Show DataFrame
cbb_df.head()

,TEAM,CONF,G,W,ADJOE,ADJDE,BARTHAG,EFG_O,EFG_D,TOR,TORD,ORB,DRB,FTR,FTRD,2P_O,2P_D,3P_O,3P_D,ADJ_T,WAB,POSTSEASON,SEED,YEAR
0,North Carolina,ACC,40,33,123.3,94.9,0.9531,52.6,48.1,15.4,18.2,40.7,30.0,32.3,30.4,53.9,44.6,32.7,36.2,71.7,8.6,2ND,1,2016
1,Wisconsin,B10,40,36,129.1,93.6,0.9758,54.8,47.7,12.4,15.8,32.1,23.7,36.2,22.4,54.8,44.7,36.5,37.5,59.3,11.3,2ND,1,2015
2,Michigan,B10,40,33,114.4,90.4,0.9375,53.9,47.7,14.0,19.5,25.5,24.9,30.7,30.0,54.7,46.8,35.2,33.2,65.9,6.9,2ND,3,2018
3,Texas Tech,B12,38,31,115.2,85.2,0.9696,53.5,43.0,17.7,22.8,27.4,28.7,32.9,36.6,52.8,41.9,36.5,29.7,67.5,7.0,2ND,3,2019
4,Gonzaga,WCC,39,37,117.8,86.3,0.9728,56.6,41.1,16.2,17.1,30.0,26.2,39.0,26.9,56.3,40.0,38.2,29.0,71.5,7.7,2ND,1,2017


In [5]:
# get list of relevant stats
allColumns = list(cbb_df.columns)
stats = allColumns[2:21]


In [6]:
# make main function to generate test scores for the particular year. Pass in theYear and stats (from dataframe header)
def cbbYearParser(theYear, stats):
  cbb_current_df = cbb_df[cbb_df.YEAR == theYear]
  cbb_current_df['POSTSEASON_BINARY'] = cbb_current_df['POSTSEASON']
  cbb_current_df['POSTSEASON_MULTI'] = cbb_current_df['POSTSEASON']

  # determine if team made it to the post season (0: no; 1: yes). Use a function
  cbb_current_df = postSeasonQuantifierBinary(cbb_current_df)
  cbb_current_df = postSeasonQuantifierMulti(cbb_current_df)

  # create an empty list to hold testScores
  allTestScores = [] ;

  #******************************************************
  # BINARY STATS:
  for stat in stats:
    testScore = statPasser_Binary(stat, cbb_current_df) ;
    allTestScores.append(testScore)

  return allTestScores, cbb_current_df


In [7]:
# make function to quantify post-season outcome
def postSeasonQuantifierBinary(cbb_current_df):
  cbb_current_df.POSTSEASON_BINARY[cbb_current_df.POSTSEASON_BINARY == "NA"] = 0
  cbb_current_df.POSTSEASON_BINARY[cbb_current_df.POSTSEASON_BINARY != 0] = 1
  
  return cbb_current_df

In [8]:
def postSeasonQuantifierMulti(cbb_current_df):
  cbb_current_df.POSTSEASON_MULTI[cbb_current_df.POSTSEASON_MULTI == "NA"] = 0
  cbb_current_df.POSTSEASON_MULTI[cbb_current_df.POSTSEASON_MULTI == "R64"] = 1
  cbb_current_df.POSTSEASON_MULTI[cbb_current_df.POSTSEASON_MULTI == "R32"] = 2
  cbb_current_df.POSTSEASON_MULTI[cbb_current_df.POSTSEASON_MULTI == "S16"] = 3
  cbb_current_df.POSTSEASON_MULTI[cbb_current_df.POSTSEASON_MULTI == "E8"] = 4
  cbb_current_df.POSTSEASON_MULTI[cbb_current_df.POSTSEASON_MULTI == "F4"] = 5
  cbb_current_df.POSTSEASON_MULTI[cbb_current_df.POSTSEASON_MULTI == "2ND"] = 6
  cbb_current_df.POSTSEASON_MULTI[cbb_current_df.POSTSEASON_MULTI == "Champions"] = 7
  
  return cbb_current_df

In [9]:
# make function to generate logistic function for stat and post-season outcome for a particular year (as defined by the 'current' df)
def statPasser_Binary(stat, cbb_current_df):

  # Assign the data to X and y (note that the X variable changes according to the 'stat' argument sent to statPasser)
  X = cbb_current_df[stat].values.reshape(-1, 1)
  y = cbb_current_df.POSTSEASON_BINARY
  y=y.astype('int')

  # from sklearn.model_selection import train_test_split
  from sklearn.model_selection import train_test_split

  # split the data
  X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

  # Create the model and fit the model to the data  
  from sklearn.linear_model import LogisticRegression

  # define the classifier 
  classifier = LogisticRegression()

  # Train the model to the data. 
  classifier.fit(X_train, y_train)

  # define variables to hold classifier scores
  trainingScore = classifier.score(X_train, y_train)
  testScore = classifier.score(X_test, y_test)

  return testScore



In [10]:
# pass each year into main function and calculate test scores.
Binary_TestScores_2015, cbb_2015 = cbbYearParser(2015, stats)
Binary_TestScores_2016, cbb_2016 = cbbYearParser(2016, stats)
Binary_TestScores_2017, cbb_2017 = cbbYearParser(2017, stats)
Binary_TestScores_2018, cbb_2018 = cbbYearParser(2018, stats)
Binary_TestScores_2019, cbb_2019 = cbbYearParser(2019, stats)


In [11]:
# make testScore dataframe
Binary_testScore_df_temp = pd.DataFrame(Binary_TestScores_2015).T
Binary_testScore_df_temp.loc[1] = Binary_TestScores_2016
Binary_testScore_df_temp.loc[2] = Binary_TestScores_2017
Binary_testScore_df_temp.loc[3] = Binary_TestScores_2018
Binary_testScore_df_temp.loc[4] = Binary_TestScores_2019

# add column headers
Binary_testScore_df_temp.columns = stats

# # make a dataframe holding the years
years = ['2015', '2016', '2017', '2018', '2019']
years_df = pd.DataFrame(years)
years_df.columns = ['Year']

# concatenate the years and stats dataframes
Binary_testScore_df = pd.concat([years_df, Binary_testScore_df_temp], axis=1).reindex(years_df.index)



In [12]:
cbb_2015.head(10)

,TEAM,CONF,G,W,ADJOE,ADJDE,BARTHAG,EFG_O,EFG_D,TOR,TORD,ORB,DRB,FTR,FTRD,2P_O,2P_D,3P_O,3P_D,ADJ_T,WAB,POSTSEASON,SEED,YEAR,POSTSEASON_BINARY,POSTSEASON_MULTI
1,Wisconsin,B10,40,36,129.1,93.6,0.9758,54.8,47.7,12.4,15.8,32.1,23.7,36.2,22.4,54.8,44.7,36.5,37.5,59.3,11.3,2ND,1,2015,1,6
5,Duke,ACC,39,35,125.2,90.6,0.9764,56.6,46.5,16.3,18.6,35.8,30.2,39.8,23.9,55.9,46.3,38.7,31.4,66.4,10.7,Champions,1,2015,1,7
10,Louisville,ACC,36,27,109.4,87.4,0.9290,47.7,44.0,17.2,21.3,34.7,30.8,38.7,33.3,48.4,43.3,30.7,30.3,65.6,5.8,E8,4,2015,1,4
11,Notre Dame,ACC,38,32,125.3,98.6,0.9401,58.3,47.9,14.5,17.3,27.9,32.2,36.7,24.1,58.2,47.4,39.0,32.6,63.9,8.6,E8,3,2015,1,4
23,Arizona,P12,38,34,117.5,86.9,0.9700,53.5,45.1,16.5,20.6,34.5,22.4,47.1,37.3,53.3,42.7,36.0,33.5,66.7,9.0,E8,2,2015,1,4
28,Gonzaga,WCC,37,34,120.2,93.1,0.9498,57.9,44.2,16.1,17.1,33.9,28.0,38.7,30.5,57.0,42.0,40.0,33.0,65.2,7.8,E8,2,2015,1,4
31,Michigan St.,B10,39,27,116.3,92.6,0.9327,53.2,44.9,17.5,16.5,33.5,27.0,32.7,39.0,50.8,43.8,38.5,31.5,63.9,3.0,F4,7,2015,1,5
37,Kentucky,SEC,39,38,120.3,84.0,0.9842,51.5,39.6,16.3,21.3,39.5,31.8,43.9,31.7,51.2,39.1,34.9,27.1,63.5,13.1,F4,1,2015,1,5
40,Duquesne,A10,30,11,107.0,111.7,0.3790,51.2,51.7,18.3,16.0,31.7,33.3,32.5,33.8,49.5,47.7,36.2,38.5,67.6,-11.3,NA,NA,2015,0,0
41,Fordham,A10,30,9,101.0,103.0,0.4450,46.7,50.2,22.2,18.7,33.4,29.5,33.1,41.7,47.8,49.6,29.8,34.1,65.9,-12.3,NA,NA,2015,0,0


In [13]:
Binary_testScore_df.head(10)

,Year,G,W,ADJOE,ADJDE,BARTHAG,EFG_O,EFG_D,TOR,TORD,ORB,DRB,FTR,FTRD,2P_O,2P_D,3P_O,3P_D,ADJ_T,WAB
0,2015,0.818182,0.909091,0.943182,0.863636,0.875000,0.818182,0.772727,0.795455,0.784091,0.784091,0.784091,0.784091,0.784091,0.806818,0.829545,0.772727,0.761364,0.784091,0.954545
1,2016,0.818182,0.852273,0.818182,0.875000,0.863636,0.818182,0.829545,0.795455,0.784091,0.784091,0.784091,0.784091,0.784091,0.784091,0.795455,0.795455,0.784091,0.784091,0.909091
2,2017,0.897727,0.920455,0.909091,0.863636,0.897727,0.795455,0.784091,0.784091,0.784091,0.784091,0.784091,0.784091,0.784091,0.795455,0.772727,0.806818,0.795455,0.784091,0.954545
3,2018,0.875000,0.886364,0.886364,0.875000,0.818182,0.818182,0.818182,0.818182,0.784091,0.784091,0.784091,0.784091,0.784091,0.772727,0.840909,0.750000,0.795455,0.784091,0.920455
4,2019,0.876404,0.898876,0.898876,0.898876,0.842697,0.786517,0.853933,0.786517,0.786517,0.786517,0.786517,0.786517,0.786517,0.797753,0.876404,0.764045,0.797753,0.786517,0.921348


In [14]:
from google.colab import files
Binary_testScore_df.to_csv('Binary_testScore.csv') 
files.download('Binary_testScore.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>